### Getting data from UWM Libraries
link: https://collections.lib.uwm.edu/digital/collection/agsphoto/search/searchterm/Still%20Image/field/type/mode/exact/conn/and

In [58]:
import requests
import pandas as pd

image_uri = lambda  id, collection: f'https://collections.lib.uwm.edu/digital/iiif/{collection}/{id}/square/360,360/0/default.jpg'
images_uri = lambda page: f'https://collections.lib.uwm.edu/digital/api/search/collection/agsafrica!agsphoto!agseurope!agsnorth!agsoceania!ags_south/page/{page}/maxRecords/2000'

data = dict(
    id= [],
    title= [],
    collection= [],
    filetype= [],
    date= []
)

page=1
max_pages=1
while True:
    response = requests.get(images_uri(page))
    if response.status_code == 200:
        resp = response.json()
    
    if not resp['items']:
        break

    for item in resp['items']:
        collection = item['collectionAlias']
        id = int(item['itemId'])
        filetype = item['filetype']
        title = item['title']

        date = None
        for row in item['metadataFields']:
            if row['field'] == 'date':
                date = row['value']
        date

        data['id'].append(id)
        data['title'].append(title)
        data['collection'].append(collection)
        data['filetype'].append(filetype)
        data['date'].append(date)

    page+=1

df = pd.DataFrame(data)
df.to_csv('all_images.csv', index=False)

### Collecting images metadata from web

### Dowloading images

In [2]:
import requests
import pandas as pd
import os
import shutil
from tqdm import tqdm

df = pd.read_csv('all_images.csv')

image_uri = lambda  id, collection: f'https://collections.lib.uwm.edu/digital/iiif/{collection}/{id}/square/360,360/0/default.jpg'
output_dir = './images'

step = 1 # must be bigger than 1
for _, row in tqdm(df.loc[1_000*(step-1):1_000*step, ['id', 'collection', 'filetype']].iterrows()):
    id, collection, filetype = row

    url = image_uri(id, collection)

    response = requests.get(url, stream=True)

    if response.status_code == 200:
        image_path = os.path.join(output_dir, f"{collection}_{id}.{filetype}")
        
        with open(image_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
    
    response.close()

1001it [28:55,  1.73s/it]
